## An overview of quality assurance practices in computational research

Authors: James Davenport, Steven Lamerton, Oliver Laslett, Vincent Knight, James Hetherington

## Introduction

- talk of different aspects of qa
  - testing
  - static analysis
- what is a test?
- why should I test?
- different types of testing as a progression
  - regression testing
  - unit testing
  - portability testing
  - performance testing

## Basic Testing

- Basic testing
- Floating point - JHD
- Testing Monte Carlo -JH
- Testing parallel code (MPI) (Use MPI4Py to make example) -JH
- Asserting on derived properties (e.g. functional derivatives)


### Floating Point and Testing

Nearly all computational research is done using the floating-point arithmetic supplied by the vendor. These days this is normally assumed to conform to the IEEE (binary) floating point system \cite{IEEE2008}, which specifies the results of a sequence of floating-point operations. This actually _does_ simplify the developers' life (compared to the days of negotiating hexadecimal-based IBM formats etc.), but does _not_ mean that there are no problems with the floating point. 



## Further Testing

* How to generate fixtures/examples (property based testing) - VK
* Regression testing with analytic solutions
* Testing for portability: how does some-one know they've installed the software correctly? JHD?
* Testing workflows
* Visualisation for QE: importance of *seeing* code
* Static analysis and linting (including dimensional and unit assertions)
* Provenance management (recipy stuff)...
* Coverage
* CI (really automated testing?)- SJL

## Testing Monte Carlo

## Testing Parallel Code

When testing distributed memory parallelisation, it we have found it helpful to write tests to validate separately the decomposition of the problem
and communication between processes, and the 

### Property based testing

In \cite{Claessen2000} a novel testing approach was described: _property based
testing_. Claessen and Hughes describe a Haskell package **QuickCheck** that
allows for the testing of functions under random inputs. In this instance it is
often not the exact output that gets tested but the "property" of the output
(thus where the name of this paradigm originates). Since this initial work some
further property based testing has been provided. In \cite{Arts2008} a mechanism
of shrinking (implemented in **Quviq QuickCheck**) of failed test cases is
described: when a given set of inputs is found that fails a test it is shrunk to
it a simplest form that still fails the test. As failing parameters are of
course reported: this aids in debugging. Other similar yet adjacent testing
frameworks are described in \cite{Runciman2008, Yang2006}, these include testing
of storage as well as exhaustive parameter testing.

In Python an implementation of _property based testing_ is implemented in the
hypothesis library \cite{hypothesis3.1}. This implements shrinking as described
above. As an example let us consider the following function which implements the
following (erroneous) property of a number that is divisible or not by 11:

> A number is divisible by 11 if and only if the alternating (in sign) sum of
> the number’s digits is 0.

As an example consider $121$, the alternating sum is: $1-2+1=0$ and indeed
$121=11\times 11$.

In [1]:
def divisible_by_11(number):
    """Uses above criterion to check if number is divisible by 11"""
    string_number = str(number)
    alternating_sum = sum([(-1) ** i * int(d) for i, d
                           in enumerate(string_number)])
    return alternating_sum == 0

In [2]:
import unittest

class TestDivisible(unittest.TestCase):
    def test_divisible_by_11(self):

        for k in range(10):
            self.assertTrue(divisible_by_11(11 * k))
            self.assertFalse(divisible_by_11(11 * k + 1))

        # Some more examples
        self.assertTrue(divisible_by_11(121))
        self.assertTrue(divisible_by_11(12122))

        self.assertFalse(divisible_by_11(123))
        self.assertFalse(divisible_by_11(12123))

TestDivisible().test_divisible_by_11()

Running the above gives no failures. Below implements a basic hypothesis test:

In [3]:
from hypothesis import given  # This is how we will define inputs
from hypothesis.strategies import integers  # This is the type of input we will use

class TestDivisible(unittest.TestCase):

    @given(k=integers(min_value=1))  # This is the main decorator
    def test_divisible_by_11(self, k):
        self.assertTrue(divisible_by_11(11 * k))

TestDivisible().test_divisible_by_11()

ImportError: No module named 'hypothesis'

An error is returned and hypothesis identifies that $k=19$ gives a failure.
Indeed: $19\times 11= 209$. This indicates that our original property for
divisibility by 11 does not hold, some basic algebra would confirm this, giving:

> A number is divisible by 11 if and only if the alternating (in sign) sum of
> the number’s digits is divisible by 11.

This can be implemented in python using:

In [ ]:
def divisible_by_11(number):
    """Uses above criterion to check if number is divisible by 11"""
    string_number = str(number)
    # Using abs as the order of the alternating sum doesn't matter.
    alternating_sum = abs(sum([(-1) ** i * int(d) for i, d
                               in enumerate(string_number)]))
    # Recursively calling the function
    return (alternating_sum in [0, 11]) or divisible_by_11(alternating_sum)

Rerunning the tests gives no failures:

In [ ]:
class TestDivisible(unittest.TestCase):

    @given(k=integers(min_value=1))  # This is the main decorator
    def test_divisible_by_11(self, k):
        self.assertTrue(divisible_by_11(11 * k))

TestDivisible().test_divisible_by_11()

## Continuous Integration

Continuous integration is a development process where code is frequently integrated on a central continuous integration server. This centralisation allows the automation of a variety of quality assurance processes such as the building of the codebase, running of tests, checking of performance and the execution of static analysis tools. By monitoring the revision control system the server can automatically run these operations as the code is changed, giving rapid feedback to the developer. Generally these systems give a web interface to view the output of the build jobs and are also extensible to allow general automation, for example the production of tarballs or other packages for formal software release. For example this paper and its associated website are built using a [continuous integration server](https://travis-ci.org/drvinceknight/TestingPaper) as changes are made to the underlying content.

A number of open source and commercial continuous integration servers are available, both hosted and for self hosting. [Travis CI](https://travis-ci.org/) is one of the most popular hosted options and has tight integration with the GitHub code repository. [Jenkins](http://jenkins-ci.org/) is the most popular of the open source, self-hosted options and has a large community writing plugins to further extend the functionality.


## Conclusions